In [1]:
import numpy as np
import pandas as pd
import polars as pl
from catboost import CatBoostClassifier, Pool, cv
import lightgbm as lgb
import xgboost as xgb
from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from functools import partial 
from sklearn.base import BaseEstimator
from scipy.stats import rankdata
from tqdm.auto import trange
from sklearn.model_selection import train_test_split, StratifiedKFold, StratifiedGroupKFold

In [33]:
mean_state_0 = np.mean([
    np.load('gemma__test_fold_0.npy'),
    np.load('gemma__test_fold_1.npy'),
    np.load('gemma__test_fold_2.npy'),
    np.load('gemma__test_fold_3.npy'),
    np.load('gemma__test_fold_4.npy'),
    np.load('IlyaGusev_gemma-2-9b-it-abliterated-test-fold_attnlrnk-0.npy'),
    #np.load('IlyaGusev_gemma-2-9b-it-abliterated-test-fold_attnlrnk-0.npy'),
    np.load('IlyaGusev_gemma-2-9b-it-abliterated-test-fold-0.npy'),  # last_token_pool
    np.load('e5_preds.npy'), np.load('e5_preds.npy'), # mean_pool
    np.load('/notebooks/BAAI-bge-multilingual-gemma2-test-fold_attnlrnk-0.npy'), # last_token_pool
    np.load('labse_en_ru_preds.npy'),# attn_pool
    np.load('IlyaGusev-gemma-2-9b-it-abliterated_mlp_old-0.npy'),
    np.load('IlyaGusev-gemma-2-9b-it-abliterated_mlp_old-0.npy'),
],axis=0)

#mean_state_1 = np.load('IlyaGusev-gemma-2-9b-it-abliterated_mlp_old-0.npy')

mean_state = mean_state_0#(mean_state_0 + mean_state_1 * 2) / 3

In [25]:
simple_sub = pd.read_csv('simple_submission.csv')

In [26]:
simple_sub

index target
0      3135    NaN
1      4655     12
2     22118      2
3     23511      0
4        45    NaN
...     ...    ...
9010   3523    NaN
9011  24925    NaN
9012   6327    NaN
9013    530    NaN
9014  20221    NaN

[9015 rows x 2 columns]

In [34]:
def prepare_predict(pred):
    preds_r = list(map(lambda y: str(round(y)),pred))
    preds_r[np.argmax(pred)] = '1'
    return ' '.join([str(i) for i,x in enumerate(preds_r) if x == '1'])

In [35]:
simple_sub['target'] = [prepare_predict(x) for x in mean_state]

In [36]:
simple_sub.to_csv('dump_subv22.csv',index=False)